In [1]:
import os

In [2]:
%pwd

'e:\\LFS\\Projects\\DS_AI_ML_DL\\NLP\\TextSummarization\\NLP_TextSummarizer\\research'

Current directory need to be changed to NLP_TextSummarizer

So, need to navigate one folder back

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\LFS\\Projects\\DS_AI_ML_DL\\NLP\\TextSummarization\\NLP_TextSummarizer'

**Python Regular Class vs Data Class**

In Python, both regular classes and data classes serve as blueprints for creating objects. However, they have different purposes and functionalities. Here's a comparison between the two:

Regular Classes:

- Regular classes are the traditional way of defining classes in Python.
- You manually define the __init__ method to initialize object attributes.
- You also need to define other special methods (e.g., __str__, __repr__, __eq__, etc.) for customizing object behavior.
Regular classes provide full flexibility and can be customized extensively.

Data Classes (introduced in Python 3.7):

- Data classes provide a concise way of declaring classes primarily meant to store data.
- They automatically generate special methods such as __init__, __repr__, __eq__, etc., based on class attributes.
- Data classes are created using the @dataclass decorator from the dataclasses module.
- They are ideal for representing data structures where you mainly need to store data and perform operations such as comparison, hashing, etc.

**Defining entity**
- Entity is nothing but the return type or the structure of the return type of a function

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Defining `ConfigurationManager` class

This will create the major artifacts folder

This will also help to retrieve the configurations of different sections like data_ingestion, training etc.

In [12]:
# importing constants => paths of config.yaml and params.yaml files
from TextSummarizer.constants import *

# importing utility functions from utils folder, common.py file
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    # constructor
    def __init__(
        self,
        # paths to yaml files for reading their content
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        
        # reading content from both the yaml files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # creating root artifacts folder
        # from config.yaml => getting artifacts_root value
        # self.config.artifacts_root is same as => self.config["artifacts_root"] 
        create_directories([self.config.artifacts_root])
    
    # method for data_ingestion 
    # return datatype is DataIngestionConfig
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # reading => data_ingestion config
        config = self.config.data_ingestion

        # creating directory using config.root_dir => artifacts/data_ingestion
        create_directories([config.root_dir])

        # constructing return details related to the data_ingestion from => config.yaml data_ingestion section
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

Now we will create the components related to data_ingestion

We will utilize the details returned by `get_data_ingestion_config` method of dtype (class) DataIngestionConfig

Creating class to handle data ingestion tasks

In [14]:
import os
# to download the zipped data file using the URL
import urllib.request as request
import zipfile
# custom logger module
from TextSummarizer.logging import logger
# custom utility function
from TextSummarizer.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
   
    # method to download the data using provided URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    # extracting the downloaded zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        # getting unzip dir path
        unzip_path = self.config.unzip_dir
        # creating unzip dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Creating the data_ingestion pipeline

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-19 19:09:24,336: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-02-19 19:09:24,392: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-19 19:09:24,403: INFO: common: created directory at: artifacts]
[2024-02-19 19:09:24,407: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-19 19:09:26,669: INFO: 2796860190: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 16A8:0939:22E59:2E37C:65D359FA
Accept-Ranges: bytes
Date: Mon, 19 Feb 2024 13:39:25 GMT
Via: 1.1 varnish
X-Served-By: cache-del21747-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1708349965.257195,VS0,VE522
Vary: Authorization,Accept-Encoding,O